In [ ]:
from torchsig.signals.signal_types import Signal
from torchsig.datasets.dataset_metadata import DatasetMetadata
from torchsig.signals.builders.constellation import ConstellationSignalBuilder
import torchsig.transforms.functional as F

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
def generate_qpsk_signal(num_iq_samples: int = 128, scale: float = 1.0) -> Signal:
    """Generate a scaled, high SNR baseband QPSK Signal.

        Args:
        num_iq_samples (int, optional): Length of sample. Defaults to 10.
        scale (int, optional): scale normalized signal data. Defaults to 1.0.

        Returns:
            signal: generated Signal 

    """
    sample_rate = 10e6
    md = DatasetMetadata(
        num_iq_samples_dataset = num_iq_samples,
        fft_size = 4,
        impairment_level = 0,
        sample_rate = sample_rate,
        num_signals_max = 1,
        num_signals_min = 1,
        num_signals_distribution = [1.0],
        snr_db_min = 100.0,
        snr_db_max = 100.0,
        signal_duration_min = 1.00*num_iq_samples/sample_rate,
        signal_duration_max = 1.00*num_iq_samples/sample_rate,
        signal_bandwidth_min = sample_rate/4,
        signal_bandwidth_max = sample_rate/4,
        signal_center_freq_min = 0.0,
        signal_center_freq_max = 0.0,         
        class_list = ['qpsk'],
        class_distribution = [1.0],
        seed = 42
    )

    builder = ConstellationSignalBuilder(
        dataset_metadata = md, 
        class_name = 'qpsk',
        seed = 42
    )
    signal = builder.build()

    # normalize, then scale data   
    signal.data = F.normalize(
        data = signal.data,
        norm_order = 2,
        flatten = False
    )
    signal.data = np.multiply(signal.data, scale)

    return signal

In [ ]:
# test data
N = 1024
qpsk_data = generate_qpsk_signal(num_iq_samples = N, scale = 1.0).data

# add in a tiny amount of noise to avoid log10(0)
complex_noise = np.sqrt(1e-6)*(np.random.normal(0,1,N) + 1j*np.random.normal(0,1,N))
qpsk_data += complex_noise

num_bits = 8
rounding_mode = 'floor'

# simulate quantization at full scale
qpsk_data_full_scale = F.quantize(
    data = qpsk_data,
    num_bits = num_bits,
    ref_level_adjustment_db = 0,
    rounding_mode=rounding_mode
)

full_scale_sfdr = np.mean(np.abs(qpsk_data)**2) / np.mean(np.abs(qpsk_data-qpsk_data_full_scale)**2)
full_scale_sfdr_db = 10*np.log10(full_scale_sfdr)
enob_full_scale = full_scale_sfdr_db/6.02

# simulate saturated ADC
qpsk_data_saturated = F.quantize(
    data = qpsk_data,
    num_bits = num_bits,
    ref_level_adjustment_db = 5,
    rounding_mode=rounding_mode
)

saturated_sfdr = np.mean(np.abs(qpsk_data)**2) / np.mean(np.abs(qpsk_data-qpsk_data_saturated)**2)
saturated_sfdr_db = 10*np.log10(saturated_sfdr)
enob_saturated = saturated_sfdr_db/6.02

# loss in dynamic range
below_level_db = -35
qpsk_data_below_full_scale = F.quantize(
    data = qpsk_data,
    num_bits = num_bits,
    ref_level_adjustment_db = below_level_db,
    rounding_mode=rounding_mode
)

below_full_scale_sfdr = np.mean(np.abs(qpsk_data)**2) / np.mean(np.abs(qpsk_data-qpsk_data_below_full_scale)**2)
below_full_scale_sfdr_db = 10*np.log10(below_full_scale_sfdr)
enob_below_full_scale = below_full_scale_sfdr_db/6.02

fig = plt.figure(figsize=(16,12))
fig.subplots_adjust(hspace=0.5)

ax = fig.add_subplot(3,1,1)
ax.plot(np.real(qpsk_data),label='Input')
ax.plot(np.real(qpsk_data_full_scale),label='Output')
ax.grid()
ax.set_title(f'Full Scale Quantization. SFDR = {full_scale_sfdr_db:0.1f} dB, ENOB = {enob_full_scale:0.1f} bits')
ax.set_xlabel('Time Index $n$',fontsize='large')
ax.set_ylabel('Amplitude')
ax.legend(fontsize='large', loc='upper left');

ax = fig.add_subplot(3,1,2)
ax.plot(np.real(qpsk_data),label='Input')
ax.plot(np.real(qpsk_data_saturated),label='Saturated Quantization')
ax.grid()
ax.set_title(f'Quantization when Saturated. SFDR = {saturated_sfdr_db:0.1f} dB, ENOB = {enob_saturated:0.1f} bits')
ax.set_xlabel('Time Index $n$',fontsize='large')
ax.set_ylabel('Amplitude')

ax = fig.add_subplot(3,1,3)
ax.plot(np.real(qpsk_data),label='Input')
ax.plot(np.real(qpsk_data_below_full_scale),label='Dynamic-Range Loss Quantization')
ax.grid()
ax.set_title(f'Quantization when Below Full Scale by {below_level_db:0.1f} dB. SFDR = {below_full_scale_sfdr_db:0.1f}, ENOB = {enob_below_full_scale:0.1f} bits')
ax.set_xlabel('Time Index $n$',fontsize='large')
ax.set_ylabel('Amplitude')
